# TensorFlow Autoencodeur avec attention pour le PAr

## Setup

Moi j'ai installé tf addons par `pip install tensorflow-addons==0.13.0` (ET NON PAS `conda install -c esri tensorflow-addons`). Voir les compatibilités [sur le github de tensorflow_addons](https://github.com/tensorflow/addons).

In [1]:
# !pip install tensorflow-addons==0.11.2

In [2]:
import sys
print(sys.version)

3.9.9 | packaged by conda-forge | (main, Dec 20 2021, 02:36:06) [MSC v.1929 64 bit (AMD64)]


In [3]:
import tensorflow as tf
import numpy as np
import io

print(tf.__version__)

2.7.0


In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
physical_devices = tf.config.list_physical_devices('GPU') 
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

Num GPUs Available:  1


# Step 1: Get the data

In [5]:
path_reglement_scol  = 'corpus.txt'
path_questions_scol  = 'touteslesquestions.txt'

# Step 2: Preprocess the data

In [6]:
import re as regex
# acquisition du texte
reglement_scol = io.open(path_reglement_scol, encoding='UTF-8').read()#.strip().split('\n')
questions_scol = io.open(path_questions_scol, encoding='UTF-8').read()#.strip().split('\n')
texte = reglement_scol + ' ' + questions_scol
texte = regex.sub("\n", " ", texte)

On crée d'abord une liste de phrases dont chaque mot est séparé par un espace. On a besoin de `spacy` pour découper correctement les mots en français d'abord.

In [7]:
import nltk
import spacy
nlp = spacy.load('fr_core_news_sm')
phrases = nltk.tokenize.sent_tokenize(texte, language='french')
print('phrases parsées par NLTK')
phrasesTokeniseesSpacy = [nlp(s) for s in phrases]
print('phrases tokénisées par spacy')
phrasesSpacy = [' '.join([token.text.lower() for token in doc]) for doc in phrasesTokeniseesSpacy]
print('phrases découpées en tokens puis refusionnées')

phrases parsées par NLTK
phrases tokénisées par spacy
phrases découpées en tokens puis refusionnées


On supprime les listes inutiles désormais

In [8]:
del phrasesTokeniseesSpacy
del phrases

Créer un tokéniseur adapté à notre vocabulaire

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(filters='')
# créer un tokenizer adapté à tout le vocabulaire des phrases
tokenizer.fit_on_texts(phrasesSpacy)

Créer les tenseurs pour toutes les phrases et padder le tout

In [10]:
tensor_sentences = tokenizer.texts_to_sequences(phrasesSpacy)
print(type(tensor_sentences))
print(phrasesSpacy[0],tensor_sentences[0])
# enfin on padd le tout pour pouvoir l'utiliser dans un réseau de neurones
tensor_sentences = tf.keras.preprocessing.sequence.pad_sequences(tensor_sentences,padding='post')

<class 'list'>
le règlement de scolarité présente les modalités d' admission à l' école centrale de lyon , les objectifs et les modalités de l' évaluation des connaissances et des compétences de la formation ingénieur , les modalités de diversification de cette formation et les conditions d' obtention des diplômes de l' école centrale de lyon , hors diplômes de master co-accrédités et diplôme d' ingénieur energie en alternance . [8, 131, 1, 59, 860, 4, 102, 6, 175, 9, 3, 25, 46, 1, 43, 10, 4, 861, 16, 4, 102, 1, 3, 77, 12, 90, 16, 12, 104, 1, 5, 42, 88, 10, 4, 102, 1, 1565, 1, 166, 42, 16, 4, 285, 6, 214, 12, 85, 1, 3, 25, 46, 1, 43, 10, 390, 85, 1, 247, 1189, 16, 22, 6, 88, 1190, 18, 615, 13]


In [11]:
# Fonction qui convertit un mot en son représentant entier
def convert(tokenizer, tensor):
    for t in tensor: # t est un entier élément du tenseur
        if t != 0:
            print ("%d ----> %s" % (t, tokenizer.index_word[t]))
convert(tokenizer, tensor_sentences[-1])

37 ----> comment
5 ----> la
38 ----> mobilité
211 ----> est-elle
1564 ----> vérifiée
15 ----> pour
4 ----> les
97 ----> doubles
85 ----> diplômes
18 ----> en
80 ----> france
2 ----> ?


# Step 3: Define problem numbers

`tokenizer.index_word` est un dictionnaire dont les clés sont des entiers et les valeurs sont des struings (mots du vocabulaire)

In [12]:
print('tensor:')
print(type(tensor_sentences))
print(np.shape(tensor_sentences))
tensor_sentences[0]
print("tokenizer:")
print(type(tokenizer))
print(type(tokenizer.index_word))

vocab_inp_size = len(tokenizer.word_index)
n_data,max_length = tensor_sentences.shape
embedding_dim = 16

print(f"nombre de données: {n_data}\nlongueur max phrases en mots: {max_length}\ntaille du vocabulaire: {vocab_inp_size}\ndimension de l'embedding: {embedding_dim}")

tensor:
<class 'numpy.ndarray'>
(2201, 347)
tokenizer:
<class 'keras_preprocessing.text.Tokenizer'>
<class 'dict'>
nombre de données: 2201
longueur max phrases en mots: 347
taille du vocabulaire: 2555
dimension de l'embedding: 16


# Step 4: Split the train and validation data

In [13]:
from sklearn.model_selection import train_test_split

# Create training and validation sets using an 80/20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(tensor_sentences, tensor_sentences, test_size=0.2)

print(type(input_tensor_train), type(target_tensor_train))
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

# on observe ce qu'il y a dans ces données: si on rééxécute ça change, c'est parce qu'il y a un shuffle aléatoire
convert(tokenizer, input_tensor_train[0])

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
1760 1760 441 441
451 ----> toute
141 ----> absence
71 ----> doit
30 ----> être
1626 ----> signalée
21 ----> par
3 ----> l'
35 ----> élève
39 ----> au
132 ----> service
1 ----> de
5 ----> la
59 ----> scolarité
8 ----> le
114 ----> plus
879 ----> tôt
56 ----> possible
16 ----> et
39 ----> au
114 ----> plus
325 ----> tard
20 ----> dans
4 ----> les
1224 ----> cinq
512 ----> jours
1046 ----> ouvrables
24 ----> qui
694 ----> suivent
8 ----> le
396 ----> premier
270 ----> jour
1 ----> de
3 ----> l'
141 ----> absence
13 ----> .


# Step 5: create Encoder and Decoder classes

In [14]:
# Encoder class
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units):
        super(Encoder, self).__init__()
        self.enc_units = enc_units


        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)


        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x):
        x = self.embedding(x)
        output, state = self.gru(x)
        return output, state
        # hidden state shape == (batch_size, hidden size)
        # output       shape == (batch_size, max_len, hidden size)

In [15]:
# Decoder class
class Decoder(tf.keras.Model):
    def __init__(self, dec_units,max_length):
        super(Decoder, self).__init__()
        self.dec_units = dec_units
        self.attention = tf.keras.layers.Attention()
        self.dense = tf.keras.layers.Dense(1)
        self.reshape = tf.keras.layers.Reshape([max_length])

    def call(self, enc_output,enc_hidden):
        attention_outputs, attention_scores = tf.keras.layers.Attention()([enc_output, enc_hidden], return_attention_scores=True)
        context = attention_outputs * enc_output
        final_output = self.dense(context)
        final_output = self.reshape(final_output)
        return final_output

encoder = Encoder(vocab_inp_size, embedding_dim, 128)
decoder = Decoder(128,10)

enc_in = tf.random.uniform(
    (6,10),
    minval=0,
    maxval=60,
    dtype=tf.dtypes.int32,
    name="dummy_input_encoder"
)


print('Encoder Input        shape: (batch_size, timesteps)                {}'.format(enc_in.shape))
enc_output, enc_hidden = encoder(enc_in)

print('Encoder Output       shape: (batch_size, sequence_length, units)   {}'.format(enc_output.shape))
print('Encoder Hidden_state shape: (batch_size, units)                    {}'.format(enc_hidden.shape))

output = decoder(enc_output)

print(output.shape)

dec_out = decoder(enc_output)
dec_out.shape
#print('Attention output: (batch_size, sequence_length, units)', attention_outputs.shape)
#print('Attention scores: (batch_size, sequence_length, units)', attention_scores.shape)

In [16]:
class Autoencoder(tf.keras.Model):
    def __init__(self, embedding_dim, vocab_inp_size, max_length, latent_dim):
        super().__init__()

        self.latent_dim = 128
        self.encoder = Encoder(vocab_inp_size, embedding_dim, latent_dim)
        self.decoder = Decoder(latent_dim,max_length)

    def call(self, inputs):
        enc_output,enc_hidden = self.encoder(inputs)
        out_dec = self.decoder(enc_output,enc_hidden)
        return out_dec
    
    def vectoriser(self, phrase):
        l = tokenizer.texts_to_sequences(phrase)
        for ll in l:
            ll += (max_length-len(ll))*[0]
            
        return(self.encoder(np.asarray(l))[0])
        


In [17]:
latent_dim = 128
autoenc = Autoencoder(embedding_dim,vocab_inp_size,max_length,128)
autoenc.compile(optimizer='Adam', loss=tf.losses.MeanSquaredError(), metrics = ["accuracy"]) # losses.MeanSquaredError() losses.CosineSimilarity()
autoenc.build(input_shape=input_tensor_train.shape)


# input_tensor_train.shape, autoenc(input_tensor_train).shape # ne pas décommenter si gros gros tenseurs

In [18]:
history = autoenc.fit(input_tensor_train,target_tensor_train,
                epochs=50,
                batch_size=64*4,
                shuffle=True,
                validation_data=(input_tensor_val,target_tensor_val),
                verbose=1)

Epoch 1/50
7/7 [==============================] - 10s 557ms/step - loss: 11286.4717 - accuracy: 0.0023 - val_loss: 11427.4580 - val_accuracy: 0.0000e+00
Epoch 2/50
7/7 [==============================] - 3s 402ms/step - loss: 11285.9648 - accuracy: 0.0000e+00 - val_loss: 11425.8242 - val_accuracy: 0.0000e+00
Epoch 3/50
3/7 [===========>..................] - ETA: 1s - loss: 10232.7529 - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [ ]:
phrase = "Comment valider en A?"

In [ ]:
autoenc.vectoriser(phrase)